# Uplift Modeling

### Importing Libraries

In [1]:
from __future__ import division
from datetime import datetime, timedelta,date
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
pyoff.init_notebook_mode()

import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report,confusion_matrix
import sklearn
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split

import warnings
warnings.filterwarnings("ignore")

In [2]:
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final


In [3]:
df_data = pd.read_csv('../datasets/market_response_model_data.csv')

In [4]:
df_data.head(10)

recency  history  used_discount  used_bogo   zip_code  is_referral channel  \
0       10   142.44              1          0  Surburban            0   Phone   
1        6   329.08              1          1      Rural            1     Web   
2        7   180.65              0          1  Surburban            1     Web   
3        9   675.83              1          0      Rural            1     Web   
4        2    45.34              1          0      Urban            0     Web   
5        6   134.83              0          1  Surburban            0   Phone   
6        9   280.20              1          0  Surburban            1   Phone   
7        9    46.42              0          1      Urban            0   Phone   
8        9   675.07              1          1      Rural            1   Phone   
9       10    32.84              0          1      Urban            1     Web   

             offer  conversion  
0  Buy One Get One           0  
1         No Offer           0  
2  Buy One Get One           0  
3         Discount           0  
4  Buy One Get One           0  
5  Buy One Get One           1  
6  Buy One Get One           0  
7  Buy One Get One           0  
8         Discount           0  
9  Buy One Get One           0

In [5]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64000 entries, 0 to 63999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recency        64000 non-null  int64  
 1   history        64000 non-null  float64
 2   used_discount  64000 non-null  int64  
 3   used_bogo      64000 non-null  int64  
 4   zip_code       64000 non-null  object 
 5   is_referral    64000 non-null  int64  
 6   channel        64000 non-null  object 
 7   offer          64000 non-null  object 
 8   conversion     64000 non-null  int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 4.4+ MB


In [6]:
df_data.conversion.mean()

0.14678125

In [7]:
def calc_uplift(df):
    avg_order_value = 25
    
    #calculate conversions for each offer type
    base_conv = df[df.offer == 'No Offer']['conversion'].mean()
    disc_conv = df[df.offer == 'Discount']['conversion'].mean()
    bogo_conv = df[df.offer == 'Buy One Get One']['conversion'].mean()
    
    #calculate conversion uplift for discount and bogo
    disc_conv_uplift = disc_conv - base_conv
    bogo_conv_uplift = bogo_conv - base_conv
    
    #calculate order uplift
    disc_order_uplift = disc_conv_uplift * len(df[df.offer == 'Discount']['conversion'])
    bogo_order_uplift = bogo_conv_uplift * len(df[df.offer == 'Buy One Get One']['conversion'])
    
    #calculate revenue uplift
    disc_rev_uplift = disc_order_uplift * avg_order_value
    bogo_rev_uplift = bogo_order_uplift * avg_order_value
    
    
    print('Discount Conversion Uplift: {0}%'.format(np.round(disc_conv_uplift*100,2)))
    print('Discount Order Uplift: {0}'.format(np.round(disc_order_uplift,2)))
    print('Discount Revenue Uplift: ${0}\n'.format(np.round(disc_rev_uplift,2)))
    
    if len(df[df.offer == 'Buy One Get One']['conversion']) > 0:
          
        print('-------------- \n')
        print('BOGO Conversion Uplift: {0}%'.format(np.round(bogo_conv_uplift*100,2)))
        print('BOGO Order Uplift: {0}'.format(np.round(bogo_order_uplift,2)))
        print('BOGO Revenue Uplift: ${0}'.format(np.round(bogo_rev_uplift,2)))     

In [8]:
calc_uplift(df_data)

Discount Conversion Uplift: 7.66%
Discount Order Uplift: 1631.89
Discount Revenue Uplift: $40797.35

-------------- 

BOGO Conversion Uplift: 4.52%
BOGO Order Uplift: 967.4
BOGO Revenue Uplift: $24185.01


In [9]:
df_data['campaign_group'] = 'treatment'
df_data.loc[df_data.offer == 'No Offer', 'campaign_group'] = 'control'

In [10]:
df_data['target_class'] = 0 #CN
df_data.loc[(df_data.campaign_group == 'control') & (df_data.conversion > 0),'target_class'] = 1 #CR
df_data.loc[(df_data.campaign_group == 'treatment') & (df_data.conversion == 0),'target_class'] = 2 #TN
df_data.loc[(df_data.campaign_group == 'treatment') & (df_data.conversion > 0),'target_class'] = 3 #TR

In [11]:
df_data.target_class.value_counts()

2    35562
0    19044
3     7132
1     2262
Name: target_class, dtype: int64

In [12]:
df_data.target_class.value_counts()/len(df_data)

2    0.555656
0    0.297563
3    0.111437
1    0.035344
Name: target_class, dtype: float64

In [13]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(df_data[['history']])
df_data['history_cluster'] = kmeans.predict(df_data[['history']])

In [14]:
df_data = order_cluster('history_cluster', 'history',df_data,True)

In [15]:
df_model = df_data.drop(['offer','campaign_group','conversion'],axis=1)
df_model = pd.get_dummies(df_model)

In [16]:
df_model.head()

recency  history  used_discount  used_bogo  is_referral  target_class  \
0       10   142.44              1          0            0             2   
1        2    45.34              1          0            0             2   
2        6   134.83              0          1            0             3   
3        9    46.42              0          1            0             2   
4       10    32.84              0          1            1             2   

   history_cluster  zip_code_Rural  zip_code_Surburban  zip_code_Urban  \
0                0               0                   1               0   
1                0               0                   0               1   
2                0               0                   1               0   
3                0               0                   0               1   
4                0               0                   0               1   

   channel_Multichannel  channel_Phone  channel_Web  
0                     0              1            0  
1                     0              0            1  
2                     0              1            0  
3                     0              1            0  
4                     0              0            1

In [17]:
#create feature set and labels
X = df_model.drop(['target_class'],axis=1)
y = df_model.target_class

In [18]:
X.columns

Index(['recency', 'history', 'used_discount', 'used_bogo', 'is_referral',
       'history_cluster', 'zip_code_Rural', 'zip_code_Surburban',
       'zip_code_Urban', 'channel_Multichannel', 'channel_Phone',
       'channel_Web'],
      dtype='object')

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=56)

In [20]:
xgb_model = xgb.XGBClassifier().fit(X_train, y_train)

In [21]:
class_probs = xgb_model.predict_proba(X_test)

In [22]:
class_probs[0]

array([0.3055568 , 0.0162501 , 0.5852461 , 0.09294704], dtype=float32)

In [23]:
X_test['proba_CN'] = class_probs[:,0] 
X_test['proba_CR'] = class_probs[:,1] 
X_test['proba_TN'] = class_probs[:,2] 
X_test['proba_TR'] = class_probs[:,3] 

In [24]:
X_test['uplift_score'] = X_test.eval('proba_CN + proba_TR - proba_TN - proba_CR')

In [25]:
X_test.head()

recency  history  used_discount  used_bogo  is_referral  \
32277        1   211.45              0          1            1   
12824        8    29.99              0          1            0   
20159        6    29.99              1          0            1   
41575       10   188.48              0          1            1   
10736       10    29.99              1          0            0   

       history_cluster  zip_code_Rural  zip_code_Surburban  zip_code_Urban  \
32277                1               0                   0               1   
12824                0               1                   0               0   
20159                0               0                   1               0   
41575                1               0                   0               1   
10736                0               0                   1               0   

       channel_Multichannel  channel_Phone  channel_Web  proba_CN  proba_CR  \
32277                     0              1            0  0.305557  0.016250   
12824                     0              0            1  0.246606  0.068676   
20159                     0              0            1  0.331185  0.023309   
41575                     0              1            0  0.304772  0.006787   
10736                     0              0            1  0.305643  0.028235   

       proba_TN  proba_TR  uplift_score  
32277  0.585246  0.092947     -0.202992  
12824  0.532680  0.152038     -0.202712  
20159  0.590403  0.055103     -0.227424  
41575  0.636702  0.051739     -0.286978  
10736  0.567770  0.098352     -0.192009

In [26]:
overall_proba = xgb_model.predict_proba(df_model.drop(['target_class'],axis=1))

In [27]:
df_model['proba_CN'] = overall_proba[:,0] 
df_model['proba_CR'] = overall_proba[:,1] 
df_model['proba_TN'] = overall_proba[:,2] 
df_model['proba_TR'] = overall_proba[:,3] 

In [28]:
df_model['uplift_score'] = df_model.eval('proba_CN + proba_TR - proba_TN - proba_CR')

In [29]:
df_data['uplift_score'] = df_model['uplift_score']

In [30]:
df_data.head()

recency  history  used_discount  used_bogo   zip_code  is_referral channel  \
0       10   142.44              1          0  Surburban            0   Phone   
1        2    45.34              1          0      Urban            0     Web   
2        6   134.83              0          1  Surburban            0   Phone   
3        9    46.42              0          1      Urban            0   Phone   
4       10    32.84              0          1      Urban            1     Web   

             offer  conversion campaign_group  target_class  history_cluster  \
0  Buy One Get One           0      treatment             2                0   
1  Buy One Get One           0      treatment             2                0   
2  Buy One Get One           1      treatment             3                0   
3  Buy One Get One           0      treatment             2                0   
4  Buy One Get One           0      treatment             2                0   

   uplift_score  
0     -0.124967  
1     -0.015485  
2     -0.068601  
3     -0.260386  
4     -0.216297

In [31]:
df_data.groupby('offer').uplift_score.mean()

offer
Buy One Get One   -0.198439
Discount          -0.196650
No Offer          -0.145709
Name: uplift_score, dtype: float32

In [32]:
df_data_lift = df_data.copy()
uplift_q_75 = df_data_lift.uplift_score.quantile(0.75)
df_data_lift = df_data_lift[(df_data_lift.offer != 'Buy One Get One') & (df_data_lift.uplift_score > uplift_q_75)].reset_index(drop=True)


In [33]:
len(df_data_lift)

11688

In [34]:
calc_uplift(df_data_lift)

Discount Conversion Uplift: 30.9%
Discount Order Uplift: 1402.72
Discount Revenue Uplift: $35067.95



In [35]:
df_data_lift = df_data.copy()
uplift_q_5 = df_data_lift.uplift_score.quantile(0.5)
df_data_lift = df_data_lift[(df_data_lift.offer != 'Buy One Get One') & (df_data_lift.uplift_score < uplift_q_5)].reset_index(drop=True)


In [36]:
calc_uplift(df_data_lift)

Discount Conversion Uplift: -3.6%
Discount Order Uplift: -417.01
Discount Revenue Uplift: $-10425.14

